<a href="https://colab.research.google.com/github/themanoftalent/pyt-colab/blob/master/turkish_mnist_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import işlemleri ile başlıyoruz


In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


In [0]:
#mnist'i bir değişkene atayalım
mnist = input_data.read_data_sets("data/MNIST/",one_hot=True)
#read_data_sets = 2 değişken girdik. dir ve one hot = 8 = [0,0,0,0,0,0,0,0,1,0]

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [0]:
#placeholder = yertutucu. 
# x = input
# y_true = test
x = tf.placeholder(tf.float32,[None,784])
y_true = tf.placeholder(tf.float32,[None,10]) # None resim sayısının istediğini tutmasını sağlar

y = x*w + b
x = input ( tanımladık )
w = weight
b = bias 
Weight ve bias eğitim esnasında optimize edilecek parametrelerdir.



In [0]:
w = tf.Variable(tf.zeros([784,10])) # Eğitilecek parametreleri tf.Variable() ile tanımlarız
# w 'nin boyutuna [784, 10] verdik. Çünkü x*w yapacağız
# x 'in boyutu [None,784] yani bir matris
# Matrislerde çarpma işlemi yapmak için ilk matrisin sutun sayısı ile ikinci matrisin satur sayısı eşit olmalı
# Bu yüzden satır sayısını 784 verdik. MNIST'in her resmi 784
# modelimiz 10 farklı tahmin yapabilir


*   x = [500,784] verdik diyelim
*   w = [784, 10] olsun
*   x*w = [500,10] yani 500 resim için ayrı ayrı 10 elemanlı tahminlerimiz olacak
*   Her resim 10 elemanlı çünkü 10 tane nöronumuz var
*   Biz burada tek layer'lı bir network oluşturuyoruz.
*   Bu layer'da 10 tane nöron olacak ve bu nöronlar resmin sınıfını tahmin edecek
* Bu matrisin içindeki her eleman şimdilik 0. Eğitim içinde bu değişerek isabet artırılacak
* Yani 500 eleman için 500 tane 10 elemanlı tahmin 
* Şimdi bias'ı tanımlayalım 






In [0]:
#Bias'ta W gibi eğitim esnasında optimize edilecek
b = tf.Variable(tf.zeros([10]))
#10 elemanımız olduğu için 10 veriyoruz.
# Bias'ımız artık 10 elemanlı bir vektör

x*w + b = [500,10] + [10]
Python'da matris ile vektör toplanıyorsa matristeki tüm elemanlar için vektör toplama işlemi gerçekleştiriliyor
500 satırımız var ve hepsi 10 elemanlı
toplama işlemi sonucu 500 elemanın hepsi 10 elemanlı bu vektör ile toplanmış olacak
**500 örnektir**


In [0]:
#Bu işleme genelde logits denir
#matrisler toplamı tf.matmul ile yapılır
logits = tf.matmul(x,w) + b

Tekrar hatırlayalım. Burada x input yani MNIST resimleri. Bu matrisin içerisinde MNIST gorsellerinin renk değerleri var. 
w ve b ise modelimizin optimize edeceği parametrelerden oluşuyor. Başlangıçta bu parametreleri zeros fonksiyonu ile tanımladığımız için hepsi sıfır. Eğitim esnasında modelimiz bu parametreleri güncelleyerek daha isabetli tahminler yapacak. 
Eğitim dediğimiz de bu zaten.
Eğitim esnasında w ve b güncellenerek, modelimiz öğretmek istediğimiz şeyi öğrenecek. 

Bu gelen logits değerini Softmax Aktivasyon Fonksiyonundan geçirerek 0 ile 1 arasına sıkıştıracağız.

Her resim için 10 uzunluğunda vektörümüz var ve bu vektörün elemanları içerisinde en büyük değer hangi elemanda ise ona göre gelen resmin hangi sınıfa ait olduğunu tahmin edecek.


In [0]:
y = tf.nn.softmax(logits=logits)
# softmax ile nörondaki değerleri sıkıştırıyoruz ve değerlerin toplamı bir'e eşit oluyor

In [0]:
#loss fonsiyonu ile modelimizin tahmini ne kadar doğru hesaplarız 
xent = tf.nn.softmax_cross_entropy_with_logits(logits = logits , labels=y_true)
# bu fonksiyon kendi softmax fonksiyonunu uyguladığı için logits'i ilk hali ile verdik 
# 2. değer olarak ise gerçek değerlerimizi verdik 
loss = tf.reduce_mean(xent) #xent'teki her değerin ortalamasını alıp loss'a atıyoruz

Bize kolaylık olması açısından modelimiz ne kadar isabetli çalışıyor onu hesaplayalım : Hesaplamamızı daha sonra yazdıracağız. İsabeti hesaplamak için önce modelimizin tahminleri ile gerçek değerleri karşılaştırıp tahminin doğru mu yoksa yanlış mı olduğunu bir değişkende saklicaz daha sonra tüm doğru ve yanlışların ortalamasını aldığımızda modelimiz ne kadar başarılı göreceğiz.
Bu doğrı ve yanlışları correct_prediction isimli bir değişkende saklayacağız.

In [0]:
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_true,1)) # argmax en büyüğün kaçta olduğunu dönderir
# Tahmin edilen ve doğru olan değer karşılaştırılacak ve True veya false bir değer döndürülecek
#correct_prediction'ın ortalamasını aldığımız zaman modelimiz yüzde kaç oranında resimleri tahmin edebiliyor göreceğiz.
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) # boolean'ı floata çevirme

In [0]:
optimize = tf.train.GradientDescentOptimizer(0.5).minimize(loss) # learning rate'i 0.5 aldık

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [0]:
batch_size = 128 # Eğitim için döngü oluşturduğunuz zaman MNIST resimlerinden her döngüde 128 tane alıp eğiteceğiz.

In [0]:
def training_step(iterations):
  for i in range(iterations):
    x_batch, y_batch = mnist.train.next_batch(batch_size=batch_size)
    feed_dict_train = {x: x_batch,y_true: y_batch}
    sess.run(optimize, feed_dict=feed_dict_train)
    

In [0]:
def test_accuracy():
  feed_dict_test = {x: mnist.test.images,y_true : mnist.test.labels}
  acc = sess.run(accuracy, feed_dict = feed_dict_test)
  print("Testing accuracy: ",acc)


In [0]:
training_step(2000)
test_accuracy()

Testing accuracy:  0.9236
